In [1]:
import torch

torch.__version__

/home/minhyuk/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.13.1+cu117'

In [1]:
from torchvision.datasets import FashionMNIST
import torch
from torchvision.transforms import ToTensor

training_dataset = FashionMNIST(
    root="res", train=True, transform=ToTensor(), download=True
)
test_dataset = FashionMNIST(
    root="res", train=False, transform=ToTensor(), download=True
)
print(training_dataset)

/home/minhyuk/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/minhyuk/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/minhyuk/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: res
    Split: Train
    StandardTransform
Transform: ToTensor()


In [3]:
from torch import nn
class Bottleneck(nn.Module):
    def __init__(self, in_channels, k) -> None:
        super().__init__()
        self.block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, k * 4, (1, 1), bias=False),
            nn.BatchNorm2d(k * 4),
            nn.ReLU(inplace=True),
            nn.Conv2d(k * 4, k, (1, 1), bias=False),
        )
        self.skip = nn.Sequential(
            nn.Identity()
        )
    
    def forward(self, x):
        x1 = self.block(x)
        x2 = self.skip(x)
        out = torch.concat((x1, x2))
        return out

Bottleneck(128, 128)

Bottleneck(
  (block): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (skip): Sequential(
    (0): Identity()
  )
)

In [4]:
class TransitionLayer(nn.Module):
    def __init__(self, in_channels) -> None:
        super().__init__()
        self.block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, in_channels, (1, 1), bias=False),
            nn.AvgPool2d((2, 2), (2, 2))
        )
    def forward(self, x):
        out = self.block(x)
        return out
    
TransitionLayer(128)

TransitionLayer(
  (block): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
  )
)

In [5]:
class MultipleDenseLayer(nn.Module):
    def __init__(self, in_channels, k, num_of_denses) -> None:
        super().__init__()
        layers = []
        for _ in range(num_of_denses):
            layers.append(Bottleneck(in_channels, k))
            in_channels += k
        self.block = nn.Sequential(
            *layers
        )
    
    def forward(self, x):
        out = self.block(x)
        return out
    
MultipleDenseLayer(32, 4, 3)

MultipleDenseLayer(
  (block): Sequential(
    (0): Bottleneck(
      (block): Sequential(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (skip): Sequential(
        (0): Identity()
      )
    )
    (1): Bottleneck(
      (block): Sequential(
        (0): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(36, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1), bia

In [6]:
class DenseNet(nn.Module):
    def __init__(self, in_channels, k, dense_blocks, num_of_classes) -> None:
        super().__init__()
        layers = []
        for i, n in enumerate(dense_blocks):
            layers.append(MultipleDenseLayer(in_channels, k, n))
            in_channels += k * n
            if i < len(dense_blocks) - 1:
                layers.append(TransitionLayer(in_channels))
        layers.append(
            nn.Sequential(
                nn.BatchNorm2d(in_channels),
                nn.ReLU(inplace=True),
                nn.AdaptiveAvgPool2d((7, 7)),
            )
        )
        self.feature_extract = nn.Sequential(*layers)
        self.classify = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7 * 7, 2048),
            nn.Linear(2048, num_of_classes)
        )
    
    def forward(self, x):
        x = self.feature_extract(x)
        out = self.classify(x)
        return out

DenseNet(32, 4, [2, 3], 10)


DenseNet(
  (feature_extract): Sequential(
    (0): MultipleDenseLayer(
      (block): Sequential(
        (0): Bottleneck(
          (block): Sequential(
            (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (4): ReLU(inplace=True)
            (5): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (skip): Sequential(
            (0): Identity()
          )
        )
        (1): Bottleneck(
          (block): Sequential(
            (0): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv2d(36, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affi